In [4]:
import shutil
import os

def delete_files_in_folder(folder_path):
    shutil.rmtree(folder_path)

delete_files_in_folder(r"C:\Users\tsx10\PythonProjectsJupyter\TUM\FP\Images\trainning\data")

# prepare folder
folder_names = ["test/0", "test/1", "train/0", "train/1"]

for name in folder_names:
    os.makedirs("../data/processed/mini MNIST npy/" + name, exist_ok=True)

for name in folder_names:
    os.makedirs("../data/raw/mini MNIST JPG/" + name, exist_ok=True)

In [5]:
# prepare the traing data

from data import split_test_train_dataset

folder_path = r"C:\Users\tsx10\PythonProjectsJupyter\TUM\FP\Images\split_images_11.06.23"
split_test_train_dataset(folder_path)

In [6]:
# preprocessing images in npy files

from preprocessing import preprocess_mini_mnist

preprocess_mini_mnist("../data/raw/mini MNIST JPG", out_dir="../data/processed/mini MNIST npy")

processing 0
processing 1
processing 0
processing 1


In [5]:
"""
minimum working example:
performs classification on mini MNIST
"""

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import json

from data import MyDataset, AddNoiseToTensor
from model import MyModel
from workflow import MyTrainer

USE_CUDA = True

def main():

    with open(r"C:\Users\tsx10\PythonProjectsJupyter\TUM\FP\Images\trainning\data\processed\mini MNIST npy\data.json") as file:
        data = json.load(file)

    # get paths and labels
    tr_paths = [sample["path"] for sample in data["train"]]
    tr_labels = [sample["label"] for sample in data["train"]]
    vl_paths = [sample["path"] for sample in data["test"]]
    vl_labels = [sample["label"] for sample in data["test"]]

    # build data augmentations
    # (torchvision transforms also provides many different pre-built augmentation and conversion transforms)
    tr_transform = transforms.Compose([
        transforms.Normalize(mean=data["info"]["mean"], std=data["info"]["std"])
        # AddNoiseToTensor(p=0.5, alpha=0.01)
    ])

    vl_transform = transforms.Compose([
        transforms.Normalize(mean=data["info"]["mean"], std=data["info"]["std"])
    ])

    # build datasets and dataloaders
    # dataloaders handle batching of samples and multiprocessing (cores defined by num_workers)
    trainset = MyDataset(paths=tr_paths, labels=tr_labels, transform=tr_transform)
    valset = MyDataset(paths=vl_paths, labels=vl_labels, transform=vl_transform)
    trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=0)
    valloader = DataLoader(valset, batch_size=1, shuffle=False, num_workers=0)

    # define model
    model = MyModel(in_channels=3, num_classes=2, channels=32, num_blocks=4)
    if USE_CUDA:
        model.cuda()  # .cuda() just pushes model to first available gpu device, alternative: model.to(<device_name>)

    # define optimizer after pushing model to device
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    criterion = torch.nn.CrossEntropyLoss(weight=None, reduction="mean")

    # define trainer
    t = MyTrainer(model=model,
                  trainloader=trainloader,
                  valloader=valloader,
                  optim=optimizer,
                  crit=criterion,
                  cuda=USE_CUDA)

    # let's gooo!
    t.train(epochs=100)
    t.save_model("../saved_weights/test_003.pt")

if __name__ == "__main__":
    main()

------------------------------------------------------------------------------------------------------------------------------------------------------
epoch 1 | tr loss: 0.565 | vl loss: 0.6215 | tr acc.: 0.7171 | vl acc.: 0.7257 | 
------------------------------------------------------------------------------------------------------------------------------------------------------
epoch 2 | tr loss: 0.551 | vl loss: 0.6436 | tr acc.: 0.7322 | vl acc.: 0.7611 | 
------------------------------------------------------------------------------------------------------------------------------------------------------
epoch 3 | tr loss: 0.5452 | vl loss: 0.6381 | tr acc.: 0.7387 | vl acc.: 0.7522 | 
------------------------------------------------------------------------------------------------------------------------------------------------------
epoch 4 | tr loss: 0.5107 | vl loss: 0.5813 | tr acc.: 0.7516 | vl acc.: 0.7345 | 
------------------------------------------------------------------

In [ ]:
# 001: in_channels=3, num_classes=2, channels=32, num_blocks=4 ||epoch 100 | tr loss: 0.2234 | vl loss: 0.261 | tr acc.: 0.9071 | vl acc.: 0.9027 | 
# 002: in_channels=3, num_classes=2, channels=64, num_blocks=4 ||epoch 100 | tr loss: 0.1616 | vl loss: 1.8543 | tr acc.: 0.9698 | vl acc.: 0.4336 |  overfitting
# 003: in_channels=3, num_classes=2, channels=32, num_blocks=6 ||epoch 100 | tr loss: 0.1735 | vl loss: 0.3269 | tr acc.: 0.9503 | vl acc.: 0.8407 | 